In [91]:
import numpy as np

import torch
from torch import optim
from torch import nn
from torch.nn import functional as F

import torchvision
from torchvision import models
from torchvision import transforms, datasets

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

### Data load

In [92]:
data = datasets.CIFAR10(root='./data', train=True, download=True)

Files already downloaded and verified


In [93]:
class MyOwnCifar(torch.utils.data.Dataset):

    def __init__(self, init_dataset, transform=None):
        self._base_dataset = init_dataset
        self.transform = transform

    def __len__(self):
        return len(self._base_dataset)

    def __getitem__(self, idx):
        img = self._base_dataset[idx][0]
        if self.transform is not None:
            img = self.transform(img)
        return img, self._base_dataset[idx][1]

In [94]:
train_transformation = transforms.Compose([transforms.Scale(256),
                                           transforms.RandomCrop(
                                               32, padding=4),
                                           transforms.ToTensor(),
                                           transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                                std=[0.229, 0.224, 0.225])])
test_transformation = transforms.Compose([transforms.ToTensor(),
                                          transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                               std=[0.229, 0.224, 0.225])])

In [95]:
train_data, test_data = train_test_split(data, test_size=0.25, random_state=42)

In [96]:
train_dataset, test_dataset = MyOwnCifar(train_data, train_transformation), MyOwnCifar(test_data, test_transformation)

In [97]:
BATCH_SIZE = 128

In [98]:
train_loader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size=BATCH_SIZE,
                                           shuffle=True,
                                           num_workers=3)
test_loader = torch.utils.data.DataLoader(test_dataset,
                                          batch_size=BATCH_SIZE,
                                          shuffle=False,
                                          num_workers=1)

In [99]:
def train_proc(dataloader, model, loss_fn, optimizer):
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        optimizer.zero_grad()
        
        # Compute prediction error
        y_pred = model(X)
        loss = loss_fn(y_pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()

    print(f'loss: {loss.item()}')

In [100]:
def test_proc(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(
        f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [101]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [102]:
EPOCHS = 10
LR = 0.01

### ImageNet Resnet-50

In [103]:
resnet_50 = models.resnet50(pretrained=True)

In [104]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
transform = transforms.Compose([transforms.Resize(
    256), transforms.CenterCrop(32), normalize, transforms.ToTensor()])

In [105]:
set_parameter_requires_grad(resnet_50, True)
resnet_50.fc = nn.Linear(2048, 100)

In [106]:
params_to_update = []
for name, param in resnet_50.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)

print(params_to_update)

[Parameter containing:
tensor([[-0.0120, -0.0014,  0.0058,  ...,  0.0165, -0.0218,  0.0029],
        [-0.0174,  0.0002,  0.0142,  ..., -0.0161, -0.0131, -0.0122],
        [ 0.0088, -0.0069,  0.0088,  ...,  0.0220, -0.0185,  0.0109],
        ...,
        [ 0.0140, -0.0192,  0.0189,  ...,  0.0069,  0.0042,  0.0111],
        [-0.0014,  0.0032, -0.0009,  ...,  0.0188, -0.0197, -0.0014],
        [ 0.0182,  0.0102, -0.0136,  ...,  0.0166, -0.0094, -0.0019]],
       requires_grad=True), Parameter containing:
tensor([-0.0039, -0.0083,  0.0100, -0.0135, -0.0063,  0.0003,  0.0042,  0.0042,
         0.0163,  0.0087, -0.0125, -0.0074, -0.0058,  0.0044,  0.0021,  0.0195,
         0.0012,  0.0074, -0.0123,  0.0160, -0.0058, -0.0022, -0.0044,  0.0011,
         0.0115, -0.0118, -0.0025, -0.0115, -0.0103,  0.0124,  0.0048,  0.0069,
        -0.0015,  0.0017, -0.0132,  0.0173,  0.0090,  0.0039,  0.0083,  0.0030,
         0.0049, -0.0003,  0.0179, -0.0009,  0.0164, -0.0163, -0.0051, -0.0076,
        -0.01

In [107]:
optimizer = torch.optim.Adam(params_to_update, lr=LR)
loss_fn = nn.CrossEntropyLoss()

In [108]:
for epoch in range(EPOCHS):
    print(f'Epoch {epoch+1}\n-------------------------------')
    train_proc(train_loader, resnet_50, loss_fn, optimizer)
    test_proc(test_loader, resnet_50, loss_fn)

Epoch 1
-------------------------------
loss: 2.9161388874053955
Test Error: 
 Accuracy: 15.7%, Avg loss: 5.261623 

Epoch 2
-------------------------------
loss: 2.970557928085327
Test Error: 
 Accuracy: 17.1%, Avg loss: 5.955867 

Epoch 3
-------------------------------
loss: 2.818028211593628
Test Error: 
 Accuracy: 13.2%, Avg loss: 6.975662 

Epoch 4
-------------------------------
loss: 2.862927198410034
Test Error: 
 Accuracy: 17.3%, Avg loss: 5.444420 

Epoch 5
-------------------------------
loss: 3.1371190547943115
Test Error: 
 Accuracy: 17.9%, Avg loss: 6.708680 

Epoch 6
-------------------------------
loss: 2.631211280822754
Test Error: 
 Accuracy: 15.8%, Avg loss: 5.399552 

Epoch 7
-------------------------------
loss: 3.3846728801727295
Test Error: 
 Accuracy: 16.4%, Avg loss: 6.009099 

Epoch 8
-------------------------------
loss: 3.246804714202881
Test Error: 
 Accuracy: 14.1%, Avg loss: 6.731711 

Epoch 9
-------------------------------
loss: 3.1674699783325195
Test

### AUGMENTATION

In [109]:
train_transformation = transforms.Compose([ 
                                    transforms.Resize(256),
                                    transforms.RandomCrop(32, padding=4),
                                     # Augmentation
                                    transforms.RandomHorizontalFlip(p=0.5),
                                    transforms.RandomVerticalFlip(p=0.5),
                                    transforms.ToTensor(),
                                    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                         std=[0.229, 0.224, 0.225])
                                    ])
test_transformation = transforms.Compose([
                                        transforms.ToTensor(),
                                       transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                         std=[0.229, 0.224, 0.225])
                                       ])

In [110]:
_train_dataset, test_dataset = MyOwnCifar(train_data, train_transformation), MyOwnCifar(test_data, test_transformation)

In [111]:
_train = torch.utils.data.ConcatDataset([train_dataset, _train_dataset])

In [112]:
_train_loader = torch.utils.data.DataLoader(_train,
                                            batch_size=BATCH_SIZE,
                                            shuffle=True,
                                            num_workers=3)
test_loader = torch.utils.data.DataLoader(test_dataset,
                                          batch_size=BATCH_SIZE,
                                          shuffle=False,
                                          num_workers=1)

In [113]:
model = models.resnet50(pretrained=True)
set_parameter_requires_grad(model, True)

In [114]:
model.fc = torch.nn.Linear(2048, 1024)
model.fc = nn.Sequential(
    torch.nn.Dropout(0.5),
    torch.nn.Linear(480, 1024),
    torch.nn.Dropout(0.2),
    torch.nn.Linear(1024, 512),
    torch.nn.Dropout(0.2),
    torch.nn.Linear(512, 256),
    torch.nn.Dropout(0.2),
    torch.nn.Linear(256, 128),
    torch.nn.Dropout(0.2),
    torch.nn.Linear(128, len(data.classes))
)

In [115]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=LR, momentum=0.9, weight_decay=5e-4)

In [116]:
params_to_update = []
for name,param in model.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)

print(params_to_update)

[Parameter containing:
tensor([[ 0.0007, -0.0175,  0.0453,  ..., -0.0311, -0.0116,  0.0240],
        [-0.0036,  0.0117, -0.0096,  ..., -0.0296,  0.0357,  0.0065],
        [ 0.0274, -0.0429, -0.0402,  ..., -0.0339,  0.0058,  0.0084],
        ...,
        [-0.0431,  0.0084,  0.0143,  ...,  0.0446,  0.0222,  0.0145],
        [-0.0417, -0.0402, -0.0009,  ...,  0.0304,  0.0439,  0.0383],
        [-0.0336, -0.0342, -0.0197,  ..., -0.0062,  0.0364,  0.0446]],
       requires_grad=True), Parameter containing:
tensor([-0.0379,  0.0270, -0.0299,  ...,  0.0450, -0.0098, -0.0175],
       requires_grad=True), Parameter containing:
tensor([[ 0.0269, -0.0300, -0.0256,  ..., -0.0115, -0.0271,  0.0179],
        [-0.0164,  0.0011, -0.0159,  ..., -0.0284, -0.0117,  0.0110],
        [-0.0284, -0.0284, -0.0195,  ..., -0.0292, -0.0202, -0.0045],
        ...,
        [ 0.0294, -0.0272,  0.0310,  ...,  0.0310, -0.0039,  0.0061],
        [-0.0161, -0.0036, -0.0107,  ...,  0.0031, -0.0259, -0.0251],
        [ 0

In [117]:
for epoch in range(EPOCHS):
    print(f'Epoch {epoch+1}\n-------------------------------')
    train_proc(_train_loader, resnet_50, loss_fn, optimizer)
    test_proc(test_loader, resnet_50, loss_fn)

Epoch 1
-------------------------------
loss: 2.513270616531372
Test Error: 
 Accuracy: 15.2%, Avg loss: 6.442940 

Epoch 2
-------------------------------
loss: 2.8124563694000244
Test Error: 
 Accuracy: 15.1%, Avg loss: 6.717832 

Epoch 3
-------------------------------
loss: 2.782818078994751
Test Error: 
 Accuracy: 15.2%, Avg loss: 6.341042 

Epoch 4
-------------------------------
loss: 3.065375804901123
Test Error: 
 Accuracy: 15.0%, Avg loss: 6.469991 

Epoch 5
-------------------------------
loss: 2.775338649749756
Test Error: 
 Accuracy: 15.3%, Avg loss: 6.535425 

Epoch 6
-------------------------------
loss: 2.503490686416626
Test Error: 
 Accuracy: 15.2%, Avg loss: 6.598553 

Epoch 7
-------------------------------
loss: 2.7860426902770996
Test Error: 
 Accuracy: 15.2%, Avg loss: 6.471208 

Epoch 8
-------------------------------
loss: 2.269205331802368
Test Error: 
 Accuracy: 15.0%, Avg loss: 6.706980 

Epoch 9
-------------------------------
loss: 2.8097572326660156
Test 

### CNN

In [118]:
class CNNNetwork(nn.Module):

    def __init__(self):
        super(CNNNetwork, self).__init__()
        
        self.dp_three = nn.Dropout(0.2)
        self.dp_four = nn.Dropout(0.2)
        
        self.bn_one = torch.nn.BatchNorm2d(3) 
        self.conv_one = torch.nn.Conv2d(3, 30, 3)
        self.bn_two = torch.nn.BatchNorm2d(30) 
        self.conv_two = torch.nn.Conv2d(30, 60, 3)
        self.bn_three = torch.nn.BatchNorm2d(60)
        self.conv_three = torch.nn.Conv2d(60, 120, 3)
        self.bn_four = torch.nn.BatchNorm2d(120)
        
        self.fc1 = torch.nn.Linear(480, 200)
        self.fc2 = torch.nn.Linear(200, 60)
        self.out = torch.nn.Linear(60, 10)
        
        
    def forward(self, x):
        
        x = self.bn_one(x)
        x = self.conv_one(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        
        x = self.bn_two(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        
        x = self.bn_three(x)
        x = self.conv_three(x)
        x = F.leaky_relu(x, 0.1)
        x = F.max_pool2d(x, 2)
        
        x = self.bn_four(x)
        x = x.view(x.size(0), -1)
        x = self.dp_three(x)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dp_four(x)
        x = self.fc2(x)
        x = F.relu(x)

        return self.out(x)

In [119]:
cnn_net = CNNNetwork()
print(cnn_net)

CNNNetwork(
  (dp_three): Dropout(p=0.2, inplace=False)
  (dp_four): Dropout(p=0.2, inplace=False)
  (bn_one): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_one): Conv2d(3, 30, kernel_size=(3, 3), stride=(1, 1))
  (bn_two): BatchNorm2d(30, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_two): Conv2d(30, 60, kernel_size=(3, 3), stride=(1, 1))
  (bn_three): BatchNorm2d(60, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_three): Conv2d(60, 120, kernel_size=(3, 3), stride=(1, 1))
  (bn_four): BatchNorm2d(120, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=480, out_features=200, bias=True)
  (fc2): Linear(in_features=200, out_features=60, bias=True)
  (out): Linear(in_features=60, out_features=10, bias=True)
)


In [120]:
optimizer = torch.optim.Adam(params_to_update, lr=LR)
loss_fn = nn.CrossEntropyLoss()

In [121]:
for epoch in range(EPOCHS):
    print(f'Epoch {epoch+1}\n-------------------------------')
    
    train_proc(train_loader, cnn_net, loss_fn, optimizer)
    test_proc(test_loader, cnn_net, loss_fn)

Epoch 1
-------------------------------
loss: 2.3262577056884766
Test Error: 
 Accuracy: 11.9%, Avg loss: 2.437292 

Epoch 2
-------------------------------
loss: 2.3280999660491943
Test Error: 
 Accuracy: 11.9%, Avg loss: 2.441877 

Epoch 3
-------------------------------
loss: 2.2863683700561523
Test Error: 
 Accuracy: 11.9%, Avg loss: 2.451318 

Epoch 4
-------------------------------
loss: 2.3234059810638428
Test Error: 
 Accuracy: 11.9%, Avg loss: 2.444909 

Epoch 5
-------------------------------
loss: 2.3065664768218994
Test Error: 
 Accuracy: 12.0%, Avg loss: 2.440886 

Epoch 6
-------------------------------
loss: 2.3495700359344482
Test Error: 
 Accuracy: 11.8%, Avg loss: 2.437615 

Epoch 7
-------------------------------
loss: 2.3231136798858643
Test Error: 
 Accuracy: 11.9%, Avg loss: 2.442049 

Epoch 8
-------------------------------
loss: 2.305312156677246
Test Error: 
 Accuracy: 11.9%, Avg loss: 2.443993 

Epoch 9
-------------------------------
loss: 2.3237202167510986
